Requirements

In [ ]:
!pip install sklearn
!pip install quandl
!pip install pandas
!pip install bs4
!pip install xgboost
!pip install sklearn
!pip install IPython
!pip install matplotlib
!pip install torch

print("Requirements Installed")

Requirements Installed


This parses the bracket using the Beautiful Soup package, and the resulting bracket is in the form of a list, named "results"

In [ ]:
from bs4 import BeautifulSoup
import requests
import pandas as pd

def getBracket(year):
  year = str(year)
  source = requests.get('https://www.sports-reference.com/cbb/postseason/'+year+'-ncaa.html').text
  soup = BeautifulSoup(source, 'lxml')

  all_rounds = soup.find_all('div', class_ = 'round')

  results = []

  for round in all_rounds:
    teams_in_round = round.find_all('div')
    counter = 0

    for matchup in teams_in_round:
      if counter%3 == 0:
        teams = matchup.find_all('div')
        for team in teams:
          info = team.a.text + " " + team.span.text
          results.append(info)
      counter+=1

  return results


This parses the analytical statistics and inserts them into a Pandas Dataframe

In [ ]:
def getStats(year):
  cols = ["Rank","Name+Seed","Conference","W-L","AdjEM","AdjO","R_AdjO","AdjD","R_AdjD","AdjT","R_AdjT","Luck","R_Luck","AdjEM1","R_AdjEM1","OppO","R_OppO","OppD","R_OppD","AdjEM2","R_AdjEM2"]
  df = pd.DataFrame([],columns = cols)

  year = str(year)
  source = requests.get('https://kenpom.com/index.php?y='+year).text
  soup = BeautifulSoup(source, 'lxml')


  all_teams = soup.find('tbody')
  all_teams = all_teams.find_all('tr')


  for team in all_teams:
    stats = team.find_all('td')
    temp_df = []
    for num in stats:
      temp_df.append(num.text)
    if len(temp_df) == 0:
      continue
    temp = pd.DataFrame([temp_df],columns = cols)
    df = df.append(temp,ignore_index=True)

  df['Win']=0

  return df

There are some inconsistencies with the naming of some teams, so this normalizes them.

In [ ]:
#(bracket,kenpom -- correct)
def cleanNames(year):
  data = pd.DataFrame()
  df = getStats(year)
  for team in getBracket(year):
    if "State" in team:
      team = team.replace("State","St.")
    if "UConn" in team:
      team = team.replace("UConn","Connecticut")
    if "NC St." in team:
      team = team.replace("NC St.","N.C. State")
    if "St. John's (NY)" in team:
      team = team.replace("St. John's (NY)","St. John's")
    if "Pitt" in team:
      team = team.replace("Pitt","Pittsburgh")
    if "UIC" in team:
      team = team.replace("UIC","Illinois Chicago")
    if "UCSB" in team:
      team = team.replace("UCSB","UC Santa Barbara")
    if "Ole Miss" in team:
      team = team.replace("Ole Miss","Mississippi")
    if "Miami (FL)" in team:
      team = team.replace("Miami (FL)","Miami FL")
    if "ETSU" in team:
      team = team.replace("ETSU","East Tennessee St.")
    if "Troy" in team:
      if year != 2017:
        team = team.replace("Troy","Troy St.")
    if "St. Joseph's" in team:
      team = team.replace("St. Joseph's","Saint Joseph's")
    if "UNC" in team:
      if "Wilmington" not in team:
        if  "Asheville" not in team:
          if "Greensboro" not in team:
            team = team.replace("UNC","North Carolina")
    if "Louisiana" in team:
      if "South" not in team:
        team = team.replace("Louisiana","Louisiana Lafayette")
    if "Alabama-Birmingham" in team:
      team = team.replace("Alabama-Birmingham","UAB")
    if "Albany (NY)" in team:
      team = team.replace("Albany (NY)","Albany")
    if "Texas A&M-Corpus Christi" in team:
      team = team.replace("Texas A&M-Corpus Christi","Texas A&M Corpus Chris")
    if "Miami (OH)" in team:
      team = team.replace("Miami (OH)","Miami OH")
    if "Texas-Arlington" in team:
      team = team.replace("Texas-Arlington","UT Arlington")
    if "Arkansas-Pine Bluff" in team:
      team = team.replace("Arkansas-Pine Bluff","Arkansas Pine Bluff")
    if "LIU" in team:
      team = team.replace("LIU","LIU Brooklyn")
    if "St. Peter's" in team:
      team = team.replace("St. Peter's","Saint Peter's")
    if "Loyola (MD)" in team:
      team = team.replace("Loyola (MD)","Loyola MD")
    if "UMass" in team:
      team = team.replace("UMass","Massachusetts")
    if "UC-Irvine" in team:
      team = team.replace("UC-Irvine","UC Irvine")
    if "Little Rock" in team:
      team = team.replace("Little Rock","Arkansas Little Rock")
    if "UC-Davis" in team:
      team = team.replace("UC-Davis","UC Davis")
    if "Loyola (IL)" in team:
      team = team.replace("Loyola (IL)","Loyola Chicago")
    if "Gardner-Webb" in team:
      team = team.replace("Gardner-Webb","Gardner Webb")
    r = df.loc[(df["Name+Seed"] == team)]
    data = data.append(r)

  return data


In [ ]:
#print(cleanNames(2019))


for year in range(2002,2020):
  if len(cleanNames(year)) !=  131:
    print("{} needs a fix by {}".format(year,131-len(cleanNames(year))))

Loops through the data structure and adds a parameter called "Win". This is the result of the game when the two adjacent teams play each other. For example, if the structure had the state [Team 1, Team 2, Team 3, Team 4], 

In [ ]:

#print(data)

def get_winners(beginning,end,data):
  data = data.reset_index(drop=True)
  pd.set_option('display.max_rows',132)
  adder = 16
  for i in range(beginning,end,2):
    if (data.iloc[i]['Name+Seed'] == data.iloc[i+adder]['Name+Seed']):
      data.at[i,'Win'] = 1
    else:
      data.at[i+1,'Win'] = 1
    adder = adder - 1
  return data


def clean_winners(data):
  if (data.iloc[124]['Name+Seed'] == data.iloc[128]['Name+Seed']):
    data.at[124,'Win'] = 1
  else:
    data.at[125,'Win'] = 1

  if (data.iloc[126]['Name+Seed'] == data.iloc[129]['Name+Seed']):
    data.at[126,'Win'] = 1
  else:
    data.at[127,'Win'] = 1

  if (data.iloc[128]['Name+Seed'] == data.iloc[130]['Name+Seed']):
    data.at[128,'Win'] = 1
  else:
    data.at[129,'Win'] = 1

  dups = [30,61,92,123,130]
  for i in dups:
    data = data.drop([i])

  data = data.reset_index(drop=True)
  pd.set_option('display.max_rows',130)

  return data

Crates two new parameters; win percentage and the seed of the team

In [ ]:


import sklearn
import numpy as np
import re
from sklearn import linear_model
from sklearn.utils import shuffle
from sklearn.preprocessing import scale, StandardScaler
from sklearn.metrics import accuracy_score
import xgboost as xgb
from sklearn.svm import SVC


def normalize_data(data,teams):
  pd.set_option('display.max_columns',None)



  winpercentages = []
  for record in range(teams):
    record = data.at[record,"W-L"]
    seperate = record.split('-')
    wins = seperate[0]
    losses = seperate[1]
    wins,losses = float(wins),float(losses)
    pctg = wins/(wins+losses)
    winpercentages.append(pctg)


  seeds = []
  for seed in range(teams):
    seed = data.at[seed,"Name+Seed"]
    match = re.findall(r'\d+',seed,flags=0)
    seeds.append(match[0])

  copy_df = data


  data['WinPercent'] = winpercentages
  data['Seeds'] = seeds


  pd.set_option('display.max_rows',130)

  return data







Scales the data and converts to tensors to prepare for training

In [ ]:
def scale_data(data):

  #removed = ["Name+Seed","Conference","W-L","AdjO","AdjD","AdjT","R_Luck","Luck","R_AdjEM1","R_AdjEM2","R_OppO","R_OppD","AdjEM","AdjEM1","OppO","OppD","AdjEM2"]
  removed = ["Name+Seed","Conference","W-L"]
  for i in removed:
    data = data.drop([i],1)
  predict = "Win"
  y = data[predict]
  y = y.astype('float32')
  data = data.drop([predict],1)

  cols = [data.columns.tolist()]
  for col in cols:
    data[col] = scale(data[col])
    data[col] = data[col].astype('float32')

  return data, y



Splits the tournament data from 2002-2019 into training and testing sets to select the best fitting model.

In [ ]:
#epochs = 2
#print(y)
#getBracket(2002)
#getStats(2002)

years = [2002,2003,2004,2005,2006,2007,2008,2009,2010,2011,2012,2013,2014,2015,2017,2018,2019]

final_data = pd.DataFrame()
final_y = pd.Series(dtype='object')

for year in years:
  data = cleanNames(year)
  data=get_winners(0,30,data)
  data=get_winners(31,61,data)
  data=get_winners(62,92,data)
  data=get_winners(93,123,data)
  data=clean_winners(data)
  data = normalize_data(data,126)
  data,y=scale_data(data)
  final_data = final_data.append(data)
  final_y = final_y.append(y)


x_train, x_test, y_train, y_test = sklearn.model_selection.train_test_split(final_data,final_y,train_size=0.8)

print("-----------------")
print("Logistic")
logistic = linear_model.LogisticRegression()
logistic.fit(x_train,y_train)
accuracy = logistic.score(x_train,y_train)
print("Training: ",accuracy)
accuracy = logistic.score(x_test,y_test)
print("Testing: ",accuracy)
#print("Co: \n",logistic.coef_)
#print("Intercept: \n", logistic.intercept_)

print("-----------------")

print("XGBoost")

x = xgb.XGBClassifier()
x.fit(x_train,y_train)
accuracy = x.score(x_train,y_train)
print("Training: ",accuracy)
accuracy = x.score(x_test,y_test)
print("Testing: ",accuracy)


print("-----------------")

print("Support Vector Machine")

vector = SVC()
vector.fit(x_train,y_train)
accuracy = vector.score(x_train,y_train)
print("Training: ",accuracy) 
accuracy = vector.score(x_test,y_test)
print("Testing: ",accuracy) 
print("\n \n \n") 

-----------------
Logistic
Training:  0.7098657326328079
Testing:  0.703962703962704
-----------------
XGBoost
Training:  0.7466433158201985
Testing:  0.6666666666666666
-----------------
Support Vector Machine
Training:  0.7157034442498541
Testing:  0.7016317016317016

 
 



In [ ]:
def my_main(year):

  data = cleanNames(year)
  data=get_winners(0,30,data)
  data=get_winners(31,61,data)
  data=get_winners(62,92,data)
  data=get_winners(93,123,data)
  data=clean_winners(data)
  data = normalize_data(data,126)
  data,y=scale_data(data)

  return data,y 


def no_clean(year):
  data = cleanNames(year)
  data=get_winners(0,30,data)
  data=get_winners(31,61,data)
  data=get_winners(62,92,data)
  data=get_winners(93,123,data)
  data=clean_winners(data)
  return data

pd.options.mode.chained_assignment = None 

pd.set_option('display.max_columns',6)
my_bracket = no_clean(2016)
og_x,y = my_main(2016)
x = logistic.predict_proba(og_x)
mine = []
for i in range(0,len(x),2):
  if x[i][1]>=x[i+1][1]:
    mine.append(1)
    mine.append(0)
  else:
    mine.append(0)
    mine.append(1)
my_bracket['Predicted Outcome'] = mine
og_x['Predicted Outcome'] = mine
####
drops = [(16,30),(46,60),(76,90),(106,126)]
for i in drops:
  for j in range(i[0],i[1]):
    my_bracket = my_bracket.drop([j])
    og_x = og_x.drop([j])
####


round_32 = my_bracket[my_bracket['Predicted Outcome']==1]
y = og_x[og_x['Predicted Outcome']==1]
y = y.drop(['Predicted Outcome'],axis=1)

x = logistic.predict_proba(y)
mine = []
for i in range(0,len(x),2):
  if x[i][1]>=x[i+1][1]:
    mine.append(1)
    mine.append(0)
  else:
    mine.append(0)
    mine.append(1)

round_32.loc[:,'Predicted Outcome']=mine
y.loc[:,'Predicted Outcome'] = mine


  

round_16 = round_32[round_32['Predicted Outcome']==1]
y = y[y['Predicted Outcome']==1]
y = y.drop(['Predicted Outcome'],axis=1)

x = logistic.predict_proba(y)
mine = []
for i in range(0,len(x),2):
  if x[i][1]>=x[i+1][1]:
    mine.append(1)
    mine.append(0)
  else:
    mine.append(0)
    mine.append(1)
round_16.loc[:,'Predicted Outcome']=mine
y.loc[:,'Predicted Outcome'] = mine




round_8 = round_16[round_16['Predicted Outcome']==1]
y = y[y['Predicted Outcome']==1]
y = y.drop(['Predicted Outcome'],axis=1)

x = logistic.predict_proba(y)
mine = []
for i in range(0,len(x),2):
  if x[i][1]>=x[i+1][1]:
    mine.append(1)
    mine.append(0)
  else:
    mine.append(0)
    mine.append(1)
round_8.loc[:,'Predicted Outcome']=mine
y.loc[:,'Predicted Outcome'] = mine






round_4 = round_8[round_8['Predicted Outcome']==1]
y = y[y['Predicted Outcome']==1]
y = y.drop(['Predicted Outcome'],axis=1)

x = logistic.predict_proba(y)
mine = []
for i in range(0,len(x),2):
  if x[i][1]>=x[i+1][1]:
    mine.append(1)
    mine.append(0)
  else:
    mine.append(0)
    mine.append(1)
round_4.loc[:,'Predicted Outcome']=mine
y.loc[:,'Predicted Outcome'] = mine




round_2 = round_4[round_4['Predicted Outcome']==1]
y = y[y['Predicted Outcome']==1]
y = y.drop(['Predicted Outcome'],axis=1)

x = logistic.predict_proba(y)
mine = []
for i in range(0,len(x),2):
  if x[i][1]>=x[i+1][1]:
    mine.append(1)
    mine.append(0)
  else:
    mine.append(0)
    mine.append(1)
round_2.loc[:,'Predicted Outcome']=mine
y.loc[:,'Predicted Outcome'] = mine

winner = round_2[round_2['Predicted Outcome']==1]


print("Round of 64")
print(my_bracket['Name+Seed'])

print("Round of 32")
print(round_32['Name+Seed'])

print("Round of 16")
print(round_16['Name+Seed'])

print("Round of 8")
print(round_8['Name+Seed'])

print("Round of 4")
print(round_4['Name+Seed'])

print("Round of 2")
print(round_2['Name+Seed'])


print("Winner")
print(winner['Name+Seed'])






#print(round_2[['Name+Seed','Predicted Outcome']])
accuracy = accuracy_score(my_bracket['Win'],my_bracket['Predicted Outcome'])
print("Resulting Accuracy: ",accuracy)

Round of 64
0             North Carolina 1
1        Florida Gulf Coast 16
2                        USC 8
3                 Providence 9
4                    Indiana 5
5               Chattanooga 12
6                   Kentucky 4
7               Stony Brook 13
8                 Notre Dame 6
9                  Michigan 11
10             West Virginia 3
11        Stephen F. Austin 14
12                 Wisconsin 7
13               Pittsburgh 10
14                    Xavier 2
15                Weber St. 15
30                    Kansas 1
31              Austin Peay 16
32                  Colorado 8
33               Connecticut 9
34                  Maryland 5
35         South Dakota St. 12
36                California 4
37                   Hawaii 13
38                   Arizona 6
39              Wichita St. 11
40                  Miami FL 3
41                  Buffalo 14
42                      Iowa 7
43                   Temple 10
44                 Villanova 2
45            UNC Asheville